In [1]:
%cd /Users/hermespuentes/dev/MINE/visual_analytics/Proyecto

In [2]:
import pandas_profiling
import math

import re
import nltk

from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.data import load
from string import punctuation

In [3]:
file = "base2.csv"

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv(file, sep=";")

In [6]:
def encodeByChar(x,encode):
    other = x.encode().decode('utf-8').encode(encode,'ignore')
    result = other.find(b'\xc3') 
    while result != -1:
        i = other[result]
        f = other[result+1]
        variable = [i,f]
        bvariable = bytes(variable)
        try:
            x = x.replace(bvariable.decode('utf-8').encode().decode(encode),bvariable.decode('utf-8'))
        except:
            x
            #print('error: '+str(bvariable))

        other = x.encode().decode('utf-8').encode(encode,'ignore')
        result = other.find(b'\xc3',result+1) 
        #print('ok '+encode+' xc3')
        
    other = x.encode().decode('utf-8').encode(encode,'ignore')
    result = other.find(b'\xc2') 
    while result != -1:
        i = other[result]
        f = other[result+1]
        variable = [i,f]
        bvariable = bytes(variable)
        try:
            x = x.replace(bvariable.decode('utf-8').encode().decode(encode),bvariable.decode('utf-8'))
        except:
            x
            #print('error: '+str(bvariable))

        other = x.encode().decode('utf-8').encode(encode,'ignore')
        result = other.find(b'\xc2',result+1) 
        #print('ok '+encode+' xc2')
    return x

In [7]:
def encodeByText(x):
    if type(x) is str:
        try:
            latin = x.encode().decode('utf-8').encode('latin-1').decode('utf-8')
            return latin
        except:
            try:
                mac = x.encode().decode('utf-8').encode('mac-turkish').decode('utf-8')
                return mac
            except:
                try:
                    cp = x.encode().decode('utf-8').encode('cp1252').decode('utf-8')
                    return cp
                except:
                    try:
                        esp = x.encode('latin-1').decode('latin-1')
                        return esp
                    except:
                        #print(x)
                        x = encodeByChar(x,'latin-1')
                        x = encodeByChar(x,'mac-turkish')
                        x = encodeByChar(x,'cp1252')
                        #print(x)
                        #print()
                        return x
    else:
        return str(x)

In [8]:
df.columns = df.columns.to_series().apply(lambda x: encodeByText(x))
df.columns

Index(['FragmentID', ' SessionEndDate', 'La historia involucra…-X',
       'La historia involucra…-Y', 'Esta historia me enseñó-X',
       'Esta historia me enseñó-Y', 'La lección impacto en…-X',
       'La lección impacto en…-Y', 'Es importante…-X', 'Es importante…-Y',
       'Sentí que tuve…-X', 'Describe algo…-X', 'Tono emocional',
       '¿Cuándo sucedieron', '¿La experiencia que compartiste es?',
       '¿Qué tan común?', '¿Por cuánto tiempo?',
       '¿Quién crees que debe saber sobre esta historia?',
       '¿En cuántos colegios/escuelas has estado?',
       'Tu historia se relaciona con_Tecnología',
       'Tu historia se relaciona con_La seguridad en el colegio',
       'Tu historia se relaciona con_Educación / aprendizaje',
       'Tu historia se relaciona con_Bullying / matoneo',
       'Tu historia se relaciona con_Comida',
       'Tu historia se relaciona con_Valores',
       'Tu historia se relaciona con_Salud',
       'Tu historia se relaciona con_Justicia',
       'Tu h

In [9]:
df['Institucion Educativa Distrital'] = df['Institucion Educativa Distrital'].apply(lambda x: encodeByText(x))
#df['Institucion Educativa Distrital']

In [10]:
df['FragmentEntry'] = df['FragmentEntry'].apply(lambda x: encodeByText(x))
#df['FragmentEntry']

In [11]:
df['¿Cuándo sucedieron'] = df['¿Cuándo sucedieron'].apply(lambda x: encodeByText(x))
#df['¿Cuándo sucedieron']

In [12]:
df['¿La experiencia que compartiste es?'] = df['¿La experiencia que compartiste es?'].apply(lambda x: encodeByText(x))
#df['¿La experiencia que compartiste es?']

In [13]:
df['¿Qué tan común?'] = df['¿Qué tan común?'].apply(lambda x: encodeByText(x))
#df['¿Qué tan común?'] 

In [14]:
df['¿Quién crees que debe saber sobre esta historia?'] = df['¿Quién crees que debe saber sobre esta historia?'].apply(lambda x: encodeByText(x))
#df['¿Quién crees que debe saber sobre esta historia?']

In [15]:
df['¿En cuántos colegios/escuelas has estado?'] = df['¿En cuántos colegios/escuelas has estado?'].apply(lambda x: encodeByText(x))
#df['¿En cuántos colegios/escuelas has estado?'] 

In [16]:
df['StoryTitle'] = df['StoryTitle'].apply(lambda x: encodeByText(x))
#df['StoryTitle'] 

In [17]:
def unionTopic(row):
    topics = ''
    if row['Tu historia se relaciona con_Tecnología'] == 1.0:
        topics = topics + "Tecnología,"
    if row['Tu historia se relaciona con_La seguridad en el colegio'] == 1.0:
        topics = topics + "La seguridad en el colegio," 
    if row['Tu historia se relaciona con_Educación / aprendizaje'] == 1.0:
        topics = topics + "Educación / aprendizaje," 
    if row['Tu historia se relaciona con_Bullying / matoneo'] == 1.0:
        topics = topics + "Bullying / matoneo," 
    if row['Tu historia se relaciona con_Comida'] == 1.0:
        topics = topics + "Comida," 
    if row['Tu historia se relaciona con_Valores'] == 1.0:
        topics = topics + "Valores," 
    if row['Tu historia se relaciona con_Salud'] == 1.0:
        topics = topics + "Salud," 
    if row['Tu historia se relaciona con_Justicia'] == 1.0:
        topics = topics + "Justicia," 
    if row['Tu historia se relaciona con_Innovación / creatividad'] == 1.0:
        topics = topics + "Innovación / creatividad," 
    if row['Tu historia se relaciona con_Relación con los compañeros'] == 1.0:
        topics = topics + "Relación con los compañeros," 
    if row['Tu historia se relaciona con_Relación con la familia'] == 1.0:
        topics = topics + "Relación con la familia," 
    if row['Tu historia se relaciona con_Relación con los profesores'] == 1.0:
        topics = topics + "Relación con los profesores," 
    if row['Tu historia se relaciona con_Ninguna de las anteriores'] == 1.0:
        topics = topics + "Ninguna de las anteriores," 
    if len(topics)>0:
        topics = topics[0:topics.rfind(',')]
    return topics

In [18]:
df['Temas'] = df.apply(unionTopic, axis=1)

In [19]:
del df['Tu historia se relaciona con_Tecnología']
del df['Tu historia se relaciona con_La seguridad en el colegio']
del df['Tu historia se relaciona con_Educación / aprendizaje']
del df['Tu historia se relaciona con_Bullying / matoneo']
del df['Tu historia se relaciona con_Comida']
del df['Tu historia se relaciona con_Valores']
del df['Tu historia se relaciona con_Salud']
del df['Tu historia se relaciona con_Justicia']
del df['Tu historia se relaciona con_Innovación / creatividad']
del df['Tu historia se relaciona con_Relación con los compañeros']
del df['Tu historia se relaciona con_Relación con la familia']
del df['Tu historia se relaciona con_Relación con los profesores']
del df['Tu historia se relaciona con_Ninguna de las anteriores']

In [20]:
def unionFamily(row):
    family = ''
    if row['¿Con quién vives?_Papá'] == 1.0:
        family = family + "Papá,"
    if row['¿Con quién vives?_mamá'] == 1.0:
        family = family + "mamá,"
    if row['¿Con quién vives?_Hermanos'] == 1.0:
        family = family + "Hermanos,"
    if row['¿Con quién vives?_otros familiares'] == 1.0:
        family = family + "otros familiares," 
    if row['¿Con quién vives?_otros'] == 1.0:
        family = family + "otros," 
    if len(family)>0:
        family = family[0:family.rfind(',')]
    return family

In [21]:
df['Familia'] = df.apply(unionFamily, axis=1)

In [22]:
del df['¿Con quién vives?_Papá']
del df['¿Con quién vives?_mamá']
del df['¿Con quién vives?_Hermanos']
del df['¿Con quién vives?_otros familiares']
del df['¿Con quién vives?_otros']

In [23]:
def unionParentStudies(row):
    parentStudies = ''
    if row['¿Con que nivel de estudios cuenta tu acudiente?_No tiene educación formal'] == 1.0:
        parentStudies = parentStudies + "No tiene educación formal,"
    if row['¿Con que nivel de estudios cuenta tu acudiente?_Primaria'] == 1.0:
        parentStudies = parentStudies + "Primaria,"
    if row['¿Con que nivel de estudios cuenta tu acudiente?_Bachiller'] == 1.0:
        parentStudies = parentStudies + "Bachiller,"
    if row['¿Con que nivel de estudios cuenta tu acudiente?_Técnico'] == 1.0:
        parentStudies = parentStudies + "Técnico," 
    if row['¿Con que nivel de estudios cuenta tu acudiente?_Pregrado (profesional)'] == 1.0:
        parentStudies = parentStudies + "Pregrado (profesional)," 
    if row['¿Con que nivel de estudios cuenta tu acudiente?_Postgrado (especialización maestría)'] == 1.0:
        parentStudies = parentStudies + "Postgrado (especialización maestría)," 
    if len(parentStudies)>0:
        parentStudies = parentStudies[0:parentStudies.rfind(',')]
    return parentStudies

In [24]:
df['EstudiosAcudiente'] = df.apply(unionParentStudies, axis=1)

In [25]:
def describe(row):
    val = row['Describe algo…-X']
    if type(val) is str:
        val = float(val.replace(',','.'))
    if math.isnan(val):
        return ""
    if val >= 0.0 and val < 0.25:
        return "Completamente novedoso"
    elif val >= 0.25 and val < 0.5:
        return "Novedoso"
    elif val >= 0.5 and val < 0.75:
        return "Pasa frecuentemente"
    else:
        return "Pasa todo el tiempo"

In [26]:
df['Describe algo que es…'] = df.apply(describe, axis=1)

In [27]:
def control(row):
    val = row['Sentí que tuve…-X']
    if type(val) is str:
        val = float(val.replace(',','.'))
    if math.isnan(val):
        return ""
    if val >= 0.0 and val < 0.25:
        return "Total control"
    elif val >= 0.25 and val < 0.5:
        return "Control"
    elif val >= 0.5 and val < 0.75:
        return "Poco control"
    else:
        return "Absolutamente ningun control"

In [28]:
df['Sentí que tuve…'] = df.apply(control, axis=1)

In [29]:
def classRoomSubjects(row):
    x = row['ST1-ClassRoomSubjects-XRight']
    y = row['ST1-ClassRoomSubjects-YTop']
    if type(x) is str:
        x = float(x.replace(',','.'))
    if type(y) is str:
        y = float(y.replace(',','.'))
    if math.isnan(x) or math.isnan(y):
        return ""
    text = ''
    if x >= 0.0 and x < 0.5:
        text = "Me interesa poco /"
    else :
        text = "Me interesa mucho /"
        
    if y >= 0.0 and y < 0.5:
        text = text + " Importa poco para mi futuro"
    else :
        text = text + " Importa mucho para mi futuro"
    return text

In [30]:
df['Las materias en clase...'] = df.apply(classRoomSubjects, axis=1)

In [31]:
def appliedLearning(row):
    x = row['ST1-AppliedLearning-XRight']
    y = row['ST1-AppliedLearning-YTop']
    if type(x) is str:
        x = float(x.replace(',','.'))
    if type(y) is str:
        y = float(y.replace(',','.'))
    if math.isnan(x) or math.isnan(y):
        return ""
    text = ''
    if x >= 0.0 and x < 0.5:
        text = "Me interesa poco /"
    else :
        text = "Me interesa mucho /"
        
    if y >= 0.0 and y < 0.5:
        text = text + " Importa poco para mi futuro"
    else :
        text = text + " Importa mucho para mi futuro"
    return text

In [32]:
df['El aprendizaje aplicado...'] = df.apply(appliedLearning, axis=1)

In [33]:
def learningThroughDiscovery(row):
    x = row['ST1-LearningThroughDiscovery-XRight']
    y = row['ST1-LearningThroughDiscovery-YTop']
    if type(x) is str:
        x = float(x.replace(',','.'))
    if type(y) is str:
        y = float(y.replace(',','.'))
    if math.isnan(x) or math.isnan(y):
        return ""
    text = ''
    if x >= 0.0 and x < 0.5:
        text = "Me interesa poco /"
    else :
        text = "Me interesa mucho /"
        
    if y >= 0.0 and y < 0.5:
        text = text + " Importa poco para mi futuro"
    else :
        text = text + " Importa mucho para mi futuro"
    return text

In [34]:
df['El aprendizaje por descubrimiento...'] = df.apply(learningThroughDiscovery, axis=1)

In [35]:
def selfLedLearning(row):
    x = row['ST1-SelfLedLearning-XRight']
    y = row['ST1-SelfLedLearning-YTop']
    if type(x) is str:
        x = float(x.replace(',','.'))
    if type(y) is str:
        y = float(y.replace(',','.'))
    if math.isnan(x) or math.isnan(y):
        return ""
    text = ''
    if x >= 0.0 and x < 0.5:
        text = "Me interesa poco /"
    else :
        text = "Me interesa mucho /"
        
    if y >= 0.0 and y < 0.5:
        text = text + " Importa poco para mi futuro"
    else :
        text = text + " Importa mucho para mi futuro"
    return text

In [36]:
df['El auto-aprendizaje...'] = df.apply(selfLedLearning, axis=1)

In [37]:
del df['¿Con que nivel de estudios cuenta tu acudiente?_No tiene educación formal']
del df['¿Con que nivel de estudios cuenta tu acudiente?_Primaria']
del df['¿Con que nivel de estudios cuenta tu acudiente?_Bachiller']
del df['¿Con que nivel de estudios cuenta tu acudiente?_Técnico']
del df['¿Con que nivel de estudios cuenta tu acudiente?_Pregrado (profesional)']
del df['¿Con que nivel de estudios cuenta tu acudiente?_Postgrado (especialización maestría)']
del df['Describe algo…-X']
del df['Sentí que tuve…-X']
del df['ST1-ClassRoomSubjects-XRight']
del df['ST1-ClassRoomSubjects-YTop']
del df['ST1-AppliedLearning-XRight']
del df['ST1-AppliedLearning-YTop']
del df['ST1-LearningThroughDiscovery-XRight']
del df['ST1-LearningThroughDiscovery-YTop']
del df['ST1-SelfLedLearning-XRight']
del df['ST1-SelfLedLearning-YTop']

In [38]:
#Definir parámetro de división
#Este parámetro es vital porque entre más cercano a 0.5, van a existir menos Neutros, entendiendo los neutros como las historias ubicadas
#hacia el centro del triángulo.
div = 0.35

In [39]:
#Todo lo mayor a esta función estará en AB
def fab(x):
    return x*math.sqrt(0.12)/0.2 - math.sqrt(0.12)

#Todo lo mayor a esta función estará en BC
def fbc(x):
    return x*-1*math.sqrt(0.12)/0.2 + 4*math.sqrt(0.12)

#Todo lo menor a este valor estará en AC
def fac():
    return math.sqrt(0.12)/2

#Todo lo mayor a este valor estará en B
def fb():
    return math.sqrt(0.75) - math.sqrt(0.12)

#Todo lo menor a esta función estará en A
def fa(x):
    return x*-1*math.sqrt(0.12)/0.2 + 2*math.sqrt(0.12)

#Todo lo menor a esta función estará en C
def fc(x):
    return x*math.sqrt(0.12)/0.2 - 3*math.sqrt(0.12)

In [40]:
def laHistoriaInvolucra(a):
    val = "Neutro"
    #The order is important!!!
    try:
        if math.isnan(float(a['La historia involucra…-Y'])) or math.isnan(fab(float(a['La historia involucra…-X']))): 
            return ""
    except ValueError:
        return "" 
    
    if float(a['La historia involucra…-Y']) >= fb(): 
        return "Familia comunidad amigos"
    
    if float(a['La historia involucra…-Y']) <= fa(float(a['La historia involucra…-X'])): 
        return "A mi"
    
    if float(a['La historia involucra…-Y']) <= fc(float(a['La historia involucra…-X'])): 
        return "Profesores"
    
    if float(a['La historia involucra…-Y']) <= fac(): 
        return "A mi/Profesores"
    
    if float(a['La historia involucra…-Y']) >= fab(float(a['La historia involucra…-X'])): 
        return "A mi/Familia comunidad amigos"
    
    if float(a['La historia involucra…-Y']) >= fbc(float(a['La historia involucra…-X'])): 
        return "Familia comunidad amigos/Profesores"
    
    return val

In [41]:
def laHistoriaMeEnsehno(a):
    val = "Neutro"
    #The order is important!!!
    try:
        if math.isnan(float(a['Esta historia me enseñó-Y'])) or math.isnan(fab(float(a['Esta historia me enseñó-X']))): 
            return ""
    except ValueError:
        return "" 
    
    if float(a['Esta historia me enseñó-Y']) >= fb(): 
        return "Comprender como son cosas"
    
    if float(a['Esta historia me enseñó-Y']) <= fa(float(a['Esta historia me enseñó-X'])): 
        return "Nuevas conocimientos habilidades"
   
    if float(a['Esta historia me enseñó-Y']) <= fc(float(a['Esta historia me enseñó-X'])): 
        return "Importancia de experimentar"
    
    if float(a['Esta historia me enseñó-Y']) <= fac(): 
        return "Nuevas conocimientos habilidades/Importancia de experimentar"
    
    if float(a['Esta historia me enseñó-Y']) >= fab(float(a['Esta historia me enseñó-X'])): 
        return "Nuevas conocimientos habilidades/Comprender como son cosas"
    
    if float(a['Esta historia me enseñó-Y']) >= fbc(float(a['Esta historia me enseñó-X'])): 
        return "Comprender como son cosas/Importancia de experimentar"
    
    return val

In [42]:
def laLeccionImpactoEn(a):
    val = "Neutro"
    #The order is important!!!
    try:
        if math.isnan(float(a['La lección impacto en…-Y'])) or math.isnan(fab(float(a['La lección impacto en…-X']))): 
            return ""
    except ValueError:
        return "" 
    
    if float(a['La lección impacto en…-Y']) >= fb(): 
        return "Oportunidades futuro"
    
    if float(a['La lección impacto en…-Y']) <= fa(float(a['La lección impacto en…-X'])): 
        return "Relación demás"
    
    if float(a['La lección impacto en…-Y']) <= fc(float(a['La lección impacto en…-X'])): 
        return "Entender sucede alrededor"
    
    if float(a['La lección impacto en…-Y']) <= fac(): 
        return "Relación demás/Entender sucede alrededor"
    
    if float(a['La lección impacto en…-Y']) >= fab(float(a['La lección impacto en…-X'])): 
        return "Relación demás/Oportunidades futuro"
    
    if float(a['La lección impacto en…-Y']) >= fbc(float(a['La lección impacto en…-X'])): 
        return "Oportunidades futuro/Entender sucede alrededor"
    
    
    return val

In [43]:
def esImportante(a):
    val = "Neutro"
    #The order is important!!!
    try:
        if math.isnan(float(a['Es importante…-Y'])) or math.isnan(fab(float(a['Es importante…-X']))): 
            return ""
    except ValueError:
        return "" 
    
    if float(a['Es importante…-Y']) >= fb(): 
        return "Ser el mejor"
    
    if float(a['Es importante…-Y']) <= fa(float(a['Es importante…-X'])): 
        return "Trabajar en equipo"
    
    if float(a['Es importante…-Y']) <= fc(float(a['Es importante…-X'])): 
        return "Comprender a los otros"
    
    if float(a['Es importante…-Y']) <= fac(): 
        return "Trabajar en equipo/Comprender a los otros"
    
    if float(a['Es importante…-Y']) >= fab(float(a['Es importante…-X'])): 
        return "Trabajar en equipo/Ser el mejor"
    
    if float(a['Es importante…-Y']) >= fbc(float(a['Es importante…-X'])): 
        return "Ser el mejor/Comprender a los otros"
    
    return val

In [44]:
df['AreaLaHistoriaInvolucra'] = df.apply(laHistoriaInvolucra, axis=1)
df['AreaEstaHistoriaMeEnseño'] = df.apply(laHistoriaMeEnsehno, axis=1)
df['AreaLaLeccionImpactoEn'] = df.apply(laLeccionImpactoEn, axis=1)
df['AreaEsImportante'] = df.apply(esImportante, axis=1)

In [45]:
del df['La historia involucra…-X']
del df['La historia involucra…-Y']
del df['Esta historia me enseñó-X']
del df['Esta historia me enseñó-Y']
del df['La lección impacto en…-X']
del df['La lección impacto en…-Y']
del df['Es importante…-X']
del df['Es importante…-Y']
#TODO Se debe manejar distinto los campos vacios

In [46]:
pandas_profiling.ProfileReport(df)

Number of variables,28
Number of observations,2152
Total Missing (%),0.0%
Total size in memory,470.8 KiB
Average record size in memory,224.0 B
Numeric,2
Categorical,25
Boolean,0
Date,0
Text (Unique),1
Rejected,0


In [47]:
nltk.download('stopwords')
nltk.download('punkt')
#lista de stopwords

spanish_stopwords = stopwords.words('spanish')
#print(spanish_stopwords)

#remover puntuacion
non_words = list(punctuation)
non_words.extend(['¿', '¡','´'])
non_words.extend(map(str,range(10)))

def tokenizar(text):
    # tokenize
    tokens =  word_tokenize(text)
    # remove stop words
    text = ' '.join([w for w in tokens if not w in spanish_stopwords])
    
    return text

def cleanText(fragment):
    text = str(fragment)
    text = text.lower()
    #Remove words with less than 2 characters
    text = re.sub(r'\b\w{1,2}\b', '', text)
    #Remove punctuation
    text = ''.join([c for c in text if c not in non_words])
    #Remove multiple spaces by one
    text = re.sub(' +', ' ', text)
    
    return tokenizar(text)
    

df['FragmentEntry'] = df['FragmentEntry'].apply(cleanText)
df['StoryTitle'] = df['StoryTitle'].apply(cleanText)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hermespuentes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hermespuentes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [48]:
df.to_csv('clean_'+file)